# How to train the Baseline Models for the SENSORIUM+ track

### This notebook will show how to
- instantiate dataloader for the Sensorium+ track
- instantiate pytorch model
- instantiate a trainer function
- train two baselines for this competition track
- save the model weights (the model weights can already be found in './model_checkpoints/pretrained/')

### Imports

In [1]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from nnfabrik.builder import get_data, get_model, get_trainer

c:\Users\manya\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Instantiate DataLoader for Sensorium+

The only difference to the Sensorium track is that here, we include the behavioral variables and the eye position,
by setting include_behavior=True, and include_eye_position=True.
this will append the behavioral variables to the input images, and the eye position will be passed to
the shifter network of the model.


In [2]:
%cd ../../

c:\Users\manya\sensorium


In [3]:
# loading the SENSORIUM+ dataset
filenames = ['notebooks/data/static27204-5-13-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip', ]


dataset_fn = 'sensorium.datasets.static_loaders'
dataset_config = {'paths': filenames,
                 'normalize': True,
                 'include_behavior': True,
                 'include_eye_position': True,
                 'batch_size': 1,
                 'scale':.25,
                 }

dataloaders = get_data(dataset_fn, dataset_config)

FileNotFoundError: [Errno 2] No such file or directory: 'notebooks/data/static27204-5-13-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip'

# Instantiate State of the Art Model (SOTA)

Because the behavioral variables are available, we instantiate the Shifter network
by setting Shifter=True in the model configuration.

In [7]:
model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
  'stack': -1,
  'layers': 4,
  'input_kern': 9,
  'gamma_input': 6.3831,
  'gamma_readout': 0.0076,
  'hidden_kern': 7,
  'hidden_channels': 64,
  'depth_separable': True,
  'grid_mean_predictor': {'type': 'cortex',
   'input_dimensions': 2,
   'hidden_layers': 1,
   'hidden_features': 30,
   'final_tanh': True},
  'init_sigma': 0.1,
  'init_mu_range': 0.3,
  'gauss_type': 'full',
  'shifter': True,
}

model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

Because the behavioral variables are available, we instantiate the Shifter network
by setting Shifter=True in the model configuration.

In [8]:
model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
  'stack': -1,
  'layers': 4,
  'input_kern': 9,
  'gamma_input': 6.3831,
  'gamma_readout': 0.0076,
  'hidden_kern': 7,
  'hidden_channels': 64,
  'depth_separable': True,
  'grid_mean_predictor': {'type': 'cortex',
   'input_dimensions': 2,
   'hidden_layers': 1,
   'hidden_features': 30,
   'final_tanh': True},
  'init_sigma': 0.1,
  'init_mu_range': 0.3,
  'gauss_type': 'full',
  'shifter': True,
}

model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

## Configure Trainer

In [11]:
trainer_fn = "sensorium.training.standard_trainer"

trainer_config = {'max_iter': 1,
                 'verbose': False,
                 'lr_decay_steps': 4,
                 'avg_loss': False,
                 'lr_init': 0.009,
                 }

trainer = get_trainer(trainer_fn=trainer_fn, 
                     trainer_config=trainer_config)

# Run model training

In [12]:
validation_score, trainer_output, state_dict = trainer(model, dataloaders, seed=42)

Epoch 1: 100%|██████████| 4471/4471 [02:31<00:00, 29.45it/s]


## Save model checkpoints

In [13]:
torch.save(model.state_dict(), './model_checkpoints/sensorium_p_sota_model.pth')

FileNotFoundError: [Errno 2] No such file or directory: './model_checkpoints/sensorium_p_sota_model.pth'

## Load Model Checkpoints

In [5]:
model.load_state_dict(torch.load("./model_checkpoints/pretrained/sensorium_p_sota_model.pth"));

---

# Train a simple LN model

In [14]:
# this will remove all nonlinearities from the CNN, and creates essentially a ln model: linear core + readout, with a subsequent non-linearity

model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
              'stack': -1,
              'layers': 3,
              'input_kern': 9,
              'gamma_input': 6.3831,
              'gamma_readout': 0.0076,
              'hidden_kern': 7,
              'hidden_channels': 64,
              'grid_mean_predictor': {'type': 'cortex',
              'input_dimensions': 2,
              'hidden_layers': 1,
              'hidden_features': 30,
              'final_tanh': True},
              'depth_separable': True,
              'init_sigma': 0.1,
              'init_mu_range': 0.3,
              'gauss_type': 'full',
              'linear': True,
              'shifter': True,
               }
model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

In [15]:
validation_score, trainer_output, state_dict = trainer(model, dataloaders, seed=42)

Epoch 1: 100%|██████████| 4471/4471 [01:19<00:00, 56.15it/s]


In [20]:
%cd c:\Users\manya\Sensorium\notebooks\model_tutorial

c:\Users\manya\Sensorium\notebooks\model_tutorial


In [21]:
torch.save(model.state_dict(), './model_checkpoints/sensorium_p_ln_model.pth')

In [7]:
model.load_state_dict(torch.load("./model_checkpoints/pretrained/sensorium_p_ln_model.pth"));

---